# Time Analysis


In [ ]:
from pyspark.sql import SparkSession

# Start Spark session
spark = SparkSession.builder.getOrCreate()

## Import Libraries


In [ ]:
from pyspark.sql import DataFrame
import pyspark.sql.functions as F
from functools import reduce
from typing import List, Union
from pyspark.sql.functions import broadcast
from pyspark.sql.window import Window
from pyspark.sql.types import TimestampType

## Read data


In [ ]:
df = spark.read.csv("Raw/data.csv", header=True, inferSchema=False, sep=";")
df = df.withColumn("Datum", F.to_timestamp("Datum", "d.M.yy h:mm"))
# drop cases with missing case_key
df = df.filter(F.col("case_key").isNotNull())

# Filter cases with event


In [ ]:
def filter_cases_by_events(
    df: DataFrame,
    case_key: str,
    event: str,
    contain_events: List[str],
    all_events_required: bool = False,
) -> DataFrame:
    """
    Filters the DataFrame to include only those case_keys that have at least one event
    matching any of the events in the contain_events list, or all events if specified.

    Parameters:
    df (DataFrame): The DataFrame containing the process data.
    case_key (str): The column name for case keys.
    event (str): The column name for events.
    contain_events (List[str]): The list of events to filter by.
    all_events_required (bool): If True, only include case_keys that contain all events in contain_events.
                                If False, include case_keys that contain at least one event in contain_events.

    Returns:
    DataFrame: The filtered DataFrame containing only the relevant case keys.

    Example:
    >>> df = spark.createDataFrame([
    ...     ("A1", "2023-01-23 08:00:00", "Antrag Start"),
    ...     ("A1", "2023-01-23 08:10:00", "Fristablauf ext. Signatul BU"),
    ...     ("A2", "2023-01-23 10:00:00", "Antrag Start"),
    ...     ("A2", "2023-01-23 10:20:00", "Sync"),
    ...     ("A2", "2023-01-23 11:00:00", "Antrag (VE)"),
    ... ], ["CASE_KEY", "Datum", "Funktion"])
    >>> contain_events = ["Antrag Start", "Sync"]
    >>> result = filter_cases_by_events(df, "CASE_KEY", "Funktion", contain_events, all_events_required=False)
    >>> result.show()
    +--------+-------------------+--------------------+
    |CASE_KEY|              Datum|            Funktion|
    +--------+-------------------+--------------------+
    |      A1|2023-01-23 08:00:00|        Antrag Start|
    |      A1|2023-01-23 08:10:00|Fristablauf ext. ...|
    |      A2|2023-01-23 10:00:00|        Antrag Start|
    |      A2|2023-01-23 10:20:00|                Sync|
    |      A2|2023-01-23 11:00:00|         Antrag (VE)|
    +--------+-------------------+--------------------+
    """

    assert isinstance(df, DataFrame), "df must be a pyspark.sql.DataFrame"
    assert case_key in df.columns, f"Column '{case_key}' not found in DataFrame"
    assert event in df.columns, f"Column '{event}' not found in DataFrame"
    assert isinstance(contain_events, list) and all(
        isinstance(e, str) for e in contain_events
    ), "contain_events should be a list of strings"

    if all_events_required:

        matching_cases = (
            df.filter(F.col(event).isin(contain_events))
            .groupBy(case_key)
            .agg(F.collect_set(event).alias("events"))
            .filter(
                F.size(
                    F.array_intersect(
                        F.col("events"), F.array(*[F.lit(e) for e in contain_events])
                    )
                )
                == len(contain_events)
            )
            .select(case_key)
        )
    else:
        matching_cases = (
            df.filter(F.col(event).isin(contain_events))
            .select(case_key)
            .dropDuplicates()
        )

    filtered_df = df.join(matching_cases, on=case_key, how="inner")

    return filtered_df

In [ ]:
def filter_cases_by_start_end_events(
    df: DataFrame,
    case_key: str,
    date_col: str,
    event_col: str,
    start_event: str,
    end_event: str,
) -> DataFrame:
    """
    Filters the DataFrame to include only those case_keys that start with the start_event
    and end with the end_event based on the date column.

    Parameters:
    df (DataFrame): The DataFrame containing the process data.
    case_key (str): The column name for case keys.
    date_col (str): The column name for dates.
    event_col (str): The column name for events.
    start_event (str): The event that should be the first event for a case.
    end_event (str): The event that should be the last event for a case.

    Returns:
    DataFrame: The filtered DataFrame containing only the relevant case keys.

    Example:
    >>> df = spark.createDataFrame([
    ...     ("A1", "2023-01-23 08:00:00", "Antrag Start"),
    ...     ("A1", "2023-01-23 08:10:00", "Fristablauf ext. Signatul BU"),
    ...     ("A1", "2023-01-23 09:00:00", "Antrag Vollstaendig"),
    ...     ("A1", "2023-01-23 10:00:00", "Geloescht"),
    ...     ("A2", "2023-01-23 10:00:00", "Antrag Start"),
    ...     ("A2", "2023-01-23 10:20:00", "Sync"),
    ...     ("A2", "2023-01-23 11:00:00", "Antrag (VE)"),
    ...     ("A2", "2023-01-23 11:01:00", "Geloescht"),
    ... ], ["CASE_KEY", "Datum", "Funktion"])
    >>> result = filter_cases_by_start_end_events(df, "CASE_KEY", "Datum", "Funktion", "Antrag Start", "Geloescht")
    >>> result.show()
    +--------+-------------------+--------------------+
    |CASE_KEY|              Datum|            Funktion|
    +--------+-------------------+--------------------+
    |      A1|2023-01-23 08:00:00|        Antrag Start|
    |      A1|2023-01-23 08:10:00|Fristablauf ext. ...|
    |      A1|2023-01-23 09:00:00|  Antrag Vollstaendig|
    |      A1|2023-01-23 10:00:00|             Geloescht|
    |      A2|2023-01-23 10:00:00|        Antrag Start|
    |      A2|2023-01-23 10:20:00|                Sync|
    |      A2|2023-01-23 11:00:00|         Antrag (VE)|
    |      A2|2023-01-23 11:01:00|             Geloescht|
    +--------+-------------------+--------------------+
    """

    # Validate input parameters
    assert isinstance(df, DataFrame), "df must be a pyspark.sql.DataFrame"
    assert case_key in df.columns, f"Column '{case_key}' not found in DataFrame"
    assert date_col in df.columns, f"Column '{date_col}' not found in DataFrame"
    assert event_col in df.columns, f"Column '{event_col}' not found in DataFrame"

    # Get the first event for each case_key
    first_events = (
        df.withColumn(
            "row_num",
            F.row_number().over(Window.partitionBy(case_key).orderBy(F.col(date_col))),
        )
        .filter(F.col("row_num") == 1)
        .filter(F.col(event_col) == start_event)
        .select(case_key)
    )

    # Get the last event for each case_key
    last_events = (
        df.withColumn(
            "row_num",
            F.row_number().over(
                Window.partitionBy(case_key).orderBy(F.col(date_col).desc())
            ),
        )
        .filter(F.col("row_num") == 1)
        .filter(F.col(event_col) == end_event)
        .select(case_key)
    )

    # Find the intersection of case_keys that have both the start_event and end_event
    matching_cases = first_events.intersect(last_events)

    # Perform an inner join to keep only the relevant case keys
    filtered_df = df.join(matching_cases, on=case_key, how="inner")

    return filtered_df

In [ ]:
def filter_cases_by_start_event(
    df: DataFrame, case_key: str, date_col: str, event_col: str, start_event: str
) -> DataFrame:
    """
    Filters the DataFrame to include only those case_keys that start with the start_event
    based on the date column.

    Parameters:
    df (DataFrame): The DataFrame containing the process data.
    case_key (str): The column name for case keys.
    date_col (str): The column name for dates.
    event_col (str): The column name for events.
    start_event (str): The event that should be the first event for a case.

    Returns:
    DataFrame: The filtered DataFrame containing only the relevant case keys.

    Example:
    >>> df = spark.createDataFrame([
    ...     ("A1", "2023-01-23 08:00:00", "Antrag Start"),
    ...     ("A1", "2023-01-23 08:10:00", "Fristablauf ext. Signatul BU"),
    ...     ("A1", "2023-01-23 09:00:00", "Antrag Vollstaendig"),
    ...     ("A1", "2023-01-23 10:00:00", "Geloescht"),
    ...     ("A2", "2023-01-23 10:00:00", "Antrag Start"),
    ...     ("A2", "2023-01-23 10:20:00", "Sync"),
    ...     ("A2", "2023-01-23 11:00:00", "Antrag (VE)"),
    ...     ("A2", "2023-01-23 11:01:00", "Geloescht"),
    ... ], ["CASE_KEY", "Datum", "Funktion"])
    >>> result = filter_cases_by_start_event(df, "CASE_KEY", "Datum", "Funktion", "Antrag Start")
    >>> result.show()
    +--------+-------------------+--------------------+
    |CASE_KEY|              Datum|            Funktion|
    +--------+-------------------+--------------------+
    |      A1|2023-01-23 08:00:00|        Antrag Start|
    |      A1|2023-01-23 08:10:00|Fristablauf ext. ...|
    |      A1|2023-01-23 09:00:00|  Antrag Vollstaendig|
    |      A1|2023-01-23 10:00:00|             Geloescht|
    |      A2|2023-01-23 10:00:00|        Antrag Start|
    |      A2|2023-01-23 10:20:00|                Sync|
    |      A2|2023-01-23 11:00:00|         Antrag (VE)|
    |      A2|2023-01-23 11:01:00|             Geloescht|
    +--------+-------------------+--------------------+
    """

    # Validate input parameters
    assert isinstance(df, DataFrame), "df must be a pyspark.sql.DataFrame"
    assert case_key in df.columns, f"Column '{case_key}' not found in DataFrame"
    assert date_col in df.columns, f"Column '{date_col}' not found in DataFrame"
    assert event_col in df.columns, f"Column '{event_col}' not found in DataFrame"

    # Get the first event for each case_key
    window_spec = Window.partitionBy(case_key).orderBy(F.col(date_col))
    first_events = (
        df.withColumn("row_num", F.row_number().over(window_spec))
        .filter(F.col("row_num") == 1)
        .filter(F.col(event_col) == start_event)
        .select(case_key)
    )

    # Perform an inner join to keep only the relevant case keys
    filtered_df = df.join(first_events, on=case_key, how="inner")

    return filtered_df

In [ ]:
def filter_cases_by_end_event(
    df: DataFrame, case_key: str, date_col: str, event_col: str, end_event: str
) -> DataFrame:
    """
    Filters the DataFrame to include only those case_keys that end with the end_event
    based on the date column.

    Parameters:
    df (DataFrame): The DataFrame containing the process data.
    case_key (str): The column name for case keys.
    date_col (str): The column name for dates.
    event_col (str): The column name for events.
    end_event (str): The event that should be the last event for a case.

    Returns:
    DataFrame: The filtered DataFrame containing only the relevant case keys.

    Example:
    >>> df = spark.createDataFrame([
    ...     ("A1", "2023-01-23 08:00:00", "Antrag Start"),
    ...     ("A1", "2023-01-23 08:10:00", "Fristablauf ext. Signatul BU"),
    ...     ("A1", "2023-01-23 09:00:00", "Antrag Vollstaendig"),
    ...     ("A1", "2023-01-23 10:00:00", "Geloescht"),
    ...     ("A2", "2023-01-23 10:00:00", "Antrag Start"),
    ...     ("A2", "2023-01-23 10:20:00", "Sync"),
    ...     ("A2", "2023-01-23 11:00:00", "Antrag (VE)"),
    ...     ("A2", "2023-01-23 11:01:00", "Geloescht"),
    ... ], ["CASE_KEY", "Datum", "Funktion"])
    >>> result = filter_cases_by_end_event(df, "CASE_KEY", "Datum", "Funktion", "Geloescht")
    >>> result.show()
    +--------+-------------------+--------------------+
    |CASE_KEY|              Datum|            Funktion|
    +--------+-------------------+--------------------+
    |      A1|2023-01-23 08:00:00|        Antrag Start|
    |      A1|2023-01-23 08:10:00|Fristablauf ext. ...|
    |      A1|2023-01-23 09:00:00|  Antrag Vollstaendig|
    |      A1|2023-01-23 10:00:00|             Geloescht|
    |      A2|2023-01-23 10:00:00|        Antrag Start|
    |      A2|2023-01-23 10:20:00|                Sync|
    |      A2|2023-01-23 11:00:00|         Antrag (VE)|
    |      A2|2023-01-23 11:01:00|             Geloescht|
    +--------+-------------------+--------------------+
    """

    # Validate input parameters
    assert isinstance(df, DataFrame), "df must be a pyspark.sql.DataFrame"
    assert case_key in df.columns, f"Column '{case_key}' not found in DataFrame"
    assert date_col in df.columns, f"Column '{date_col}' not found in DataFrame"
    assert event_col in df.columns, f"Column '{event_col}' not found in DataFrame"

    # Get the last event for each case_key
    window_spec = Window.partitionBy(case_key).orderBy(F.col(date_col).desc())
    last_events = (
        df.withColumn("row_num", F.row_number().over(window_spec))
        .filter(F.col("row_num") == 1)
        .filter(F.col(event_col) == end_event)
        .select(case_key)
    )

    # Perform an inner join to keep only the relevant case keys
    filtered_df = df.join(last_events, on=case_key, how="inner")

    return filtered_df

In [ ]:
def filter_cases_by_timeframe(
    df: DataFrame, case_key: str, date_col: str, timeframe: str, time_duration: int = 1
) -> DataFrame:
    """
    Filters the DataFrame to include only those case_keys that started and ended
    within the defined timeframe.

    Parameters:
    df (DataFrame): The DataFrame containing the process data.
    case_key (str): The column name for case keys.
    date_col (str): The column name for dates.
    timeframe (str): The timeframe to filter by ('hour', 'day', 'week', 'month', 'year').
    time_duration (int): The duration of the timeframe to filter by (default is 1).

    Returns:
    DataFrame: The filtered DataFrame containing only the relevant case keys.

    Example:
    >>> df = spark.createDataFrame([
    ...     ("A1", "2023-01-23 08:00:00"),
    ...     ("A1", "2023-01-23 08:10:00"),
    ...     ("A1", "2023-01-23 09:00:00"),
    ...     ("A1", "2023-01-23 10:00:00"),
    ...     ("A2", "2023-01-23 10:00:00"),
    ...     ("A2", "2023-01-23 10:20:00"),
    ...     ("A2", "2023-01-23 11:00:00"),
    ...     ("A2", "2023-01-23 11:01:00"),
    ... ], ["CASE_KEY", "Datum"])
    >>> result = filter_cases_by_timeframe(df, "CASE_KEY", "Datum", "hour", 1)
    >>> result.show()
    +--------+-------------------+
    |CASE_KEY|              Datum|
    +--------+-------------------+
    |      A2|2023-01-23 10:00:00|
    |      A2|2023-01-23 10:20:00|
    |      A2|2023-01-23 11:00:00|
    |      A2|2023-01-23 11:01:00|
    +--------+-------------------+
    """

    # Validate input parameters
    assert isinstance(df, DataFrame), "df must be a pyspark.sql.DataFrame"
    assert case_key in df.columns, f"Column '{case_key}' not found in DataFrame"
    assert date_col in df.columns, f"Column '{date_col}' not found in DataFrame"
    assert timeframe in [
        "hour",
        "day",
        "week",
        "month",
        "year",
    ], "timeframe must be one of 'hour', 'day', 'week', 'month', 'year'"

    # Convert the date column to TimestampType if it is not already
    df = df.withColumn(date_col, F.col(date_col).cast(TimestampType()))

    # Calculate the first and last event times for each case_key
    window_spec = Window.partitionBy(case_key).orderBy(F.col(date_col))
    first_last_times = (
        df.withColumn("first_event_time", F.first(
            F.col(date_col)).over(window_spec))
        .withColumn("last_event_time", F.last(F.col(date_col)).over(window_spec))
        .select(case_key, "first_event_time", "last_event_time")
        .distinct()
    )

    # Define the time difference condition based on the timeframe
    time_diff_cond = {
        "hour": F.expr(
            f"first_event_time + INTERVAL {time_duration} HOUR >= last_event_time"
        ),
        "day": F.expr(
            f"first_event_time + INTERVAL {time_duration} DAY >= last_event_time"
        ),
        "week": F.expr(
            f"first_event_time + INTERVAL {time_duration} WEEK >= last_event_time"
        ),
        "month": F.expr(
            f"first_event_time + INTERVAL {time_duration} MONTH >= last_event_time"
        ),
        "year": F.expr(
            f"first_event_time + INTERVAL {time_duration} YEAR >= last_event_time"
        ),
    }

    # Filter the case_keys that satisfy the timeframe condition
    matching_cases = first_last_times.filter(
        time_diff_cond[timeframe]).select(case_key)

    # Perform an inner join to keep only the relevant case keys
    filtered_df = df.join(matching_cases, on=case_key, how="inner")

    return filtered_df

In [ ]:
def filter_cases_with_concurrent_events(
    df: DataFrame, case_key: str, date_col: str, event_col: str
) -> DataFrame:
    """
    Filters the DataFrame to include only those case_keys that have multiple events
    occurring at the same exact timestamp.

    Parameters:
    df (DataFrame): The DataFrame containing the process data.
    case_key (str): The column name for case keys.
    date_col (str): The column name for dates.
    event_col (str): The column name for events.

    Returns:
    DataFrame: The filtered DataFrame containing only the relevant case keys.

    Example:
    >>> df = spark.createDataFrame([
    ...     ("A1", "2023-01-23 08:00:00", "Eingang"),
    ...     ("A1", "2023-01-23 08:00:00", "Los"),
    ...     ("A1", "2023-01-23 09:00:00", "Berechnung"),
    ...     ("A2", "2023-01-23 10:00:00", "Eingang"),
    ...     ("A2", "2023-01-23 11:00:00", "Berechnung"),
    ...     ("A2", "2023-01-23 11:00:00", "Abschluss"),
    ...     ("A2", "2023-01-23 11:00:00", "Versand"),
    ... ], ["CASE_KEY", "Datum", "Funktion"])
    >>> result = filter_cases_with_concurrent_events(df, "CASE_KEY", "Datum", "Funktion")
    >>> result.show()
    +--------+-------------------+---------+
    |CASE_KEY|              Datum| Funktion|
    +--------+-------------------+---------+
    |      A1|2023-01-23 08:00:00|  Eingang|
    |      A1|2023-01-23 08:00:00|      Los|
    |      A2|2023-01-23 11:00:00|Berechnung|
    |      A2|2023-01-23 11:00:00|Abschluss|
    |      A2|2023-01-23 11:00:00|  Versand|
    +--------+-------------------+---------+
    """

    # Validate input parameters
    assert isinstance(df, DataFrame), "df must be a pyspark.sql.DataFrame"
    assert case_key in df.columns, f"Column '{case_key}' not found in DataFrame"
    assert date_col in df.columns, f"Column '{date_col}' not found in DataFrame"
    assert event_col in df.columns, f"Column '{event_col}' not found in DataFrame"

    # Define the window specification to partition by case_key and date_col
    window_spec = Window.partitionBy(case_key, date_col)

    # Count the number of events at each timestamp for each case_key
    df_with_counts = df.withColumn(
        "event_count", F.count(event_col).over(window_spec))

    # Filter the DataFrame to include only rows where the event_count is greater than 1
    concurrent_events_df = df_with_counts.filter(F.col("event_count") > 1)

    # Drop the event_count column as it is no longer needed
    result_df = concurrent_events_df.drop("event_count")

    return result_df

In [ ]:
def filter_cases_by_start_and_next_event(
    df: DataFrame,
    case_key: str,
    date_col: str,
    event_col: str,
    start_event: str,
    next_event: str,
) -> DataFrame:
    """
    Filters the DataFrame to include only those case_keys that start with the start_event
    and are directly followed by the next_event based on the date column.

    Parameters:
    df (DataFrame): The DataFrame containing the process data.
    case_key (str): The column name for case keys.
    date_col (str): The column name for dates.
    event_col (str): The column name for events.
    start_event (str): The event that should be the first event for a case.
    next_event (str): The event that should directly follow the start_event for a case.

    Returns:
    DataFrame: The filtered DataFrame containing only the relevant case keys.
    """

    # Validate input parameters
    assert isinstance(df, DataFrame), "df must be a pyspark.sql.DataFrame"
    assert case_key in df.columns, f"Column '{case_key}' not found in DataFrame"
    assert date_col in df.columns, f"Column '{date_col}' not found in DataFrame"
    assert event_col in df.columns, f"Column '{event_col}' not found in DataFrame"

    # Get the first event for each case_key
    window_spec = Window.partitionBy(case_key).orderBy(F.col(date_col))
    first_events = (
        df.withColumn("row_num", F.row_number().over(window_spec))
        .filter(F.col("row_num") == 1)
        .filter(F.col(event_col) == start_event)
        .select(case_key)
    )

    # Get the second event for each case_key
    second_events = (
        df.withColumn("row_num", F.row_number().over(window_spec))
        .filter(F.col("row_num") == 2)
        .filter(F.col(event_col) == next_event)
        .select(case_key)
    )

    # Perform an inner join to keep only the relevant case keys
    filtered_df = df.join(first_events, on=case_key, how="inner").join(
        second_events, on=case_key, how="inner"
    )

    return filtered_df

In [ ]:
def filter_cases_by_start_and_never_followed_event(
    df: DataFrame,
    case_key: str,
    date_col: str,
    event_col: str,
    start_event: str,
    never_event: str,
) -> DataFrame:
    """
    Filters the DataFrame to include only those case_keys that start with the start_event
    and are never followed by the never_event based on the date column.

    Parameters:
    df (DataFrame): The DataFrame containing the process data.
    case_key (str): The column name for case keys.
    date_col (str): The column name for dates.
    event_col (str): The column name for events.
    start_event (str): The event that should be the first event for a case.
    never_event (str): The event that should never follow the start_event for a case.

    Returns:
    DataFrame: The filtered DataFrame containing only the relevant case keys.
    """

    # Validate input parameters
    assert isinstance(df, DataFrame), "df must be a pyspark.sql.DataFrame"
    assert case_key in df.columns, f"Column '{case_key}' not found in DataFrame"
    assert date_col in df.columns, f"Column '{date_col}' not found in DataFrame"
    assert event_col in df.columns, f"Column '{event_col}' not found in DataFrame"

    # Define a window partitioned by case_key and ordered by date_col
    window_spec = Window.partitionBy(case_key).orderBy(F.col(date_col))

    # Add a column 'next_event' which shows the next event for each row
    df = df.withColumn("next_event", F.lead(event_col).over(window_spec))

    # Filter rows where event_col is start_event and next_event is not never_event
    filtered_df = df.filter(
        (F.col(event_col) == start_event) & (
            F.col("next_event") != never_event)
    )

    return filtered_df

In [ ]:
def filter_cases_by_never_flowed_through_event(
    df: DataFrame, case_key: str, event_col: str, never_events: list
) -> DataFrame:
    """
    Filters the DataFrame to include only those case_keys that never flowed through the never_events.

    Parameters:
    df (DataFrame): The DataFrame containing the process data.
    case_key (str): The column name for case keys.
    event_col (str): The column name for events.
    never_events (list): The list of events that should never occur for a case.

    Returns:
    DataFrame: The filtered DataFrame containing only the relevant case keys.
    """

    # Validate input parameters
    assert isinstance(df, DataFrame), "df must be a pyspark.sql.DataFrame"
    assert case_key in df.columns, f"Column '{case_key}' not found in DataFrame"
    assert event_col in df.columns, f"Column '{event_col}' not found in DataFrame"
    assert isinstance(never_events, list), "never_events must be a list"
    assert all(
        isinstance(event, str) for event in never_events
    ), "All elements in never_events must be strings"

    # Create a DataFrame of cases that went through the never_events
    never_df = (
        df.filter(F.col(event_col).isin(never_events)).select(case_key).distinct()
    )

    # Perform a left anti join to find cases that never went through the never_events
    filtered_df = df.join(never_df, on=case_key, how="left_anti")

    return filtered_df